In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import datajoint as dj
import tensorflow as tf
import numpy as np

import seaborn as sns
sns.set_style('ticks', rc={'image.cmap': 'bwr'})

import os
import sys
import inspect
import scipy

p = !pwd
p = os.path.dirname(os.path.dirname(p[0]))
if p not in sys.path:
    sys.path.append(p)

In [ ]:
from cnn_sys_ident.mesonet.data import MultiDataset
from cnn_sys_ident.mesonet.parameters import Core, Readout, Model, RegPath, Fit
from cnn_sys_ident.mesonet import MODELS

In [ ]:
data_key = dict(data_hash='cfcd208495d565ef66e7dff9f98764da')
model_rel = MODELS['HermiteTransfer'] * MultiDataset() & data_key

# Populate database

In [ ]:
# Will run for a long time if executed from within the notebook on a single GPU
# We distribute it among a GPU cluster
#
# Fit().populate(model_rel, reserve_jobs=True, suppress_errors=True, order='random')

# Print results

In [ ]:
k_transfer = 2
keys = model_rel.fetch(dj.key)
rel = Fit() * Readout.SpatialXFeatureJointL1Transfer() & keys & dict(k_transfer=k_transfer)
test_corr, key = rel.fetch('test_corr', dj.key, order_by='val_loss', limit=1)
key = key[0]
print('Core trained on 1/{:d} of the neurons'.format(k_transfer))
print('Average correlation of best model on test set: {:.3f}'.format(test_corr[0]))

In [ ]:
model = Fit().load_model(key)
num_neurons = model.base.data.num_neurons

In [ ]:
data = model.base.data
inputs, responses = data.test()
feed_dict = {model.base.inputs: inputs, model.base.is_training: False}
predictions = model.base.evaluate(model.predictions, feed_dict=feed_dict)

idx_train = np.arange(0, num_neurons, k_transfer)
idx_test = np.setdiff1d(np.arange(num_neurons), idx_train)

rho_test = np.array([scipy.stats.pearsonr(p, r)[0] for p, r in zip(
    predictions[:,idx_test].T, responses[:,idx_test].T)])
rho_test[predictions[:,idx_test].std(axis=0) < 1e-4] = 0
rho_train = np.array([scipy.stats.pearsonr(p, r)[0] for p, r in zip(
    predictions[:,idx_train].T, responses[:,idx_train].T)])
rho_train[predictions[:,idx_train].std(axis=0) < 1e-4] = 0
print('Average correlation of neurons:\n  - used to train core: {:.3f}\n  - modeled via transfer learning: {:.3f}'.format(
    rho_train[~np.isnan(rho_train)].mean(), rho_test[~np.isnan(rho_test)].mean()))